# * Sales Data

In [1]:
import configparser
import datetime as dt
import pandas as pd
import numpy as np
import xlrd
import oracledb
import re
import FN_Monitoring as fn

config = configparser.ConfigParser()
config.read('../../my_config.ini')
config.sections()

pd.set_option('future.no_silent_downcasting', True)

TDMDBPR_user = config['TDMDBPR']['username']
TDMDBPR_pwd = config['TDMDBPR']['password']
TDMDBPR_db = config['TDMDBPR']['db']
TDMDBPR_host = config['TDMDBPR']['host']
TDMDBPR_port = config['TDMDBPR']['port']

AKPIPRD_user = config['AKPIPRD']['username']
AKPIPRD_pwd = config['AKPIPRD']['password']
AKPIPRD_db = config['AKPIPRD']['db']
AKPIPRD_host = config['AKPIPRD']['host']
AKPIPRD_port = config['AKPIPRD']['port']

curr_dt = dt.datetime.now().date()
str_curr_dt = curr_dt.strftime('%Y%m%d')

## Actual : Monitoring
->  DIM_CORP_KPI

In [2]:
''' Execute Summary Data '''


# Input parameter
curr_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
v_start_date = 20240101
print(f'\nParameter input...\n')
print(f'   -> v_start_date: {v_start_date}')


# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
print(f'\n{TDMDBPR_db} : Connected')
src_cur = src_conn.cursor()
query = (f"""
    SELECT /*+PARALLEL(8)*/
        SUBSTR(TM_KEY_DAY,1,6) AS TM_KEY_MTH, PRODUCT, COMP_CD, METRIC_CD, METRIC_NAME
        , MAX(LOAD_DATE) LOAD_DATE, MIN(TM_KEY_DAY) AS START_DAY, MAX(TM_KEY_DAY) AS END_DAY
        , SUM(CASE WHEN AREA_TYPE = 'C' THEN METRIC_VALUE END) C
        , SUM(CASE WHEN AREA_TYPE = 'P' THEN METRIC_VALUE END) P
        , SUM(CASE WHEN AREA_TYPE = 'G' THEN METRIC_VALUE END) G
        , SUM(CASE WHEN AREA_TYPE = 'H' THEN METRIC_VALUE END) H
        , SUM(CASE WHEN AREA_TYPE = 'HH' THEN METRIC_VALUE END) HH
        , SUM(CASE WHEN AREA_TYPE = 'CCAA' THEN METRIC_VALUE END) CCAA
        , SUM(CASE WHEN AREA_TYPE = 'CCAATT' THEN METRIC_VALUE END) CCAATT
        , COUNT(DISTINCT AREA_TYPE) CNT_AREA_TYPE, COUNT(DISTINCT AREA_CD) CNT_AREA_CD, COUNT(1) ROW_CNT
    FROM CDSAPPO.DIM_CORP_KPI
    WHERE REGEXP_LIKE(METRIC_CD, 'CS$|CG$') -->> 2025 KPIs
    AND EXISTS (
        SELECT 1 -->> AREA_CD IN (P, G, H, R, HH, CCAA, CCAATT)
        FROM (
            SELECT DISTINCT ORGID_P AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT ORGID_G AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT ORGID_R AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT ORGID_H AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT ORGID_HH AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT SUBSTR(CCAATT,1,4) AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT CCAATT AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
        ) TMP
        WHERE TMP_CD = AREA_CD )
    AND TM_KEY_DAY >= {v_start_date}
    GROUP BY SUBSTR(TM_KEY_DAY,1,6), PRODUCT, COMP_CD, METRIC_CD, METRIC_NAME
""")


try:
    execute_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
    print(f'\n   -> Execute query... {execute_datetime}')

    # Get : Summary DataFrame
    src_cur.execute(query)
    rows = src_cur.fetchall()
    chk_src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])
    print(f'\n   -> Summary DataFrame : {chk_src_df.shape[0]} rows, {chk_src_df.shape[1]} columns')

    src_cur.close()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()
    print(f'\n{TDMDBPR_db} : Disconnected')


Parameter input...

   -> v_start_date: 20240101

TDMDBPR : Connected

   -> Execute query... 2025-06-20, 10:47:16

   -> Summary DataFrame : 1268 rows, 18 columns

TDMDBPR : Disconnected


In [3]:
def my_metric_group(v_name):

    # Get : Parameter
    name = v_name
    result = ''
    
    if re.search('Gross Adds', name): result = 'Gross Adds'
    elif re.search('Inflow M1', name): result = 'Inflow M1'
    elif re.search('Inflow M2', name): result = 'Inflow M2'
    elif re.search('No of AP 1D', name): result = 'No of AP 1D'
    elif re.search('Amt of AP 1D', name): result = 'Amt of AP 1D'
    elif re.search('No of AP In Month', name): result = 'No of AP In Month'
    elif re.search('Amt of AP In Month', name): result = 'Amt of AP In Month'
    # elif re.search('CConnected : onsumer', name): result = 'Gross Adds - Con - Consumer'
    # elif re.search('Connected : DataService', name): result = 'Gross Adds - Con - DataService'
    # elif re.search('Registration : Consumer', name): result = 'Gross Adds - Reg - Consumer'
    # elif re.search('Registration : DataService', name): result = 'Gross Adds - Reg - DataService'
    else: result = 'Unknown' 
    
    return result

In [4]:
''' Add Columns '''

chk_src_df['CHANNEL_GRP'] = np.where(chk_src_df['METRIC_CD'].str.contains('CS$'), 'ALL', 'GEO')
chk_src_df['LOCATION_GRP'] = np.where(chk_src_df['METRIC_NAME'].str.contains('Location'), 'Customer', 'Sales')
chk_src_df['METRIC_GRP'] = chk_src_df.apply(lambda x: my_metric_group(str(x['METRIC_NAME'])), axis=1)
# tmp_df['MAIN_FLAG'] = np.where((tmp_df['METRIC_CD'].str.contains('[0-9]C$|[0-9]H$|[0-9]MCOM$|[0-9]CUS$')) | (tmp_df['METRIC_NAME'].str.contains('New Revenue|Existing Revenue')), 'N', 'Y')
# chk_src_df.tail()

# chk_src_df.loc[chk_src_df['PRODUCT']=='TOL'].tail()

### KPIs Summary

In [5]:
''' PREPAID Summary : 14 rows, 24 KPIs '''

curr_mth = chk_src_df['TM_KEY_MTH'].max()
product = 'PREPAID'
# channel_grp = 'GEO'

prepaid_df = chk_src_df.copy()
prepaid_df = prepaid_df.loc[prepaid_df['TM_KEY_MTH']==curr_mth]
prepaid_df = prepaid_df.loc[prepaid_df['PRODUCT']==product]
# prepaid_df = prepaid_df.loc[prepaid_df['CHANNEL_GRP']==channel_grp]

prepaid_df['ALL'] = prepaid_df['P']
prepaid_df['TRUE'] = np.where(prepaid_df['COMP_CD']=='TRUE', prepaid_df['P'], 0)
prepaid_df['DTAC'] = np.where(prepaid_df['COMP_CD']=='DTAC', prepaid_df['P'], 0)
prepaid_df['TMP_NAME'] = prepaid_df['METRIC_NAME'].replace(r' : DTAC| : TMH', '', regex=True)

prepaid_df = prepaid_df.groupby(['PRODUCT', 'METRIC_GRP', 'TMP_NAME']).agg({'LOAD_DATE':'max', 'START_DAY':'min', 'END_DAY':'max', 'ALL':'sum', 'TRUE':'sum', 'DTAC':'sum'})
prepaid_df = prepaid_df.fillna(0).sort_values(by=['PRODUCT', 'METRIC_GRP', 'TMP_NAME']).reset_index()

mod_col_list = prepaid_df.iloc[:, 6:9].columns.tolist()
for col in mod_col_list:
    prepaid_df[col] = prepaid_df[col].apply(lambda x: format(x, ',.0f'))

prepaid_df

,PRODUCT,METRIC_GRP,TMP_NAME,LOAD_DATE,START_DAY,END_DAY,ALL,TRUE,DTAC
0,PREPAID,Amt of AP 1D,Prepaid Amt of AP 1D,2025-06-20 08:02:16,20250601,20250619,"195,689,331","94,128,784","101,560,547"
1,PREPAID,Amt of AP 1D,Prepaid Amt of AP 1D - GEO Channel,2025-06-20 08:02:16,20250601,20250619,"192,229,272","92,930,033","99,299,239"
2,PREPAID,Amt of AP In Month,Prepaid Amt of AP In Month,2025-06-20 08:02:16,20250601,20250619,"207,380,574","99,051,065","108,329,509"
3,PREPAID,Amt of AP In Month,Prepaid Amt of AP In Month - GEO Channel,2025-06-20 08:02:16,20250601,20250619,"203,746,690","97,835,528","105,911,162"
4,PREPAID,Gross Adds,Prepaid Gross Adds,2025-06-20 08:02:16,20250601,20250619,"1,033,578","480,014","553,564"
5,PREPAID,Gross Adds,Prepaid Gross Adds - GEO Channel,2025-06-20 08:02:16,20250601,20250619,"1,019,833","478,332","541,501"
6,PREPAID,Inflow M1,Prepaid Inflow M1,2025-06-20 08:02:16,20250601,20250618,"201,051,460","89,615,029","111,436,431"
7,PREPAID,Inflow M1,Prepaid Inflow M1 - GEO Channel,2025-06-20 08:02:16,20250601,20250618,"197,600,658","88,518,785","109,081,873"
8,PREPAID,Inflow M2,Prepaid Inflow M2,2025-06-20 08:02:16,20250601,20250618,"82,698,458","37,457,564","45,240,894"
9,PREPAID,Inflow M2,Prepaid Inflow M2 - GEO Channel,2025-06-20 08:02:16,20250601,20250618,"81,817,060","37,394,812","44,422,248"


In [6]:
''' POSTPAID Summary : 8 rows, 16 KPIs '''

curr_mth = chk_src_df['TM_KEY_MTH'].max()
product = 'POSTPAID'
# channel_grp = 'GEO'

postpaid_df = chk_src_df.copy()
postpaid_df = postpaid_df.loc[postpaid_df['TM_KEY_MTH']==curr_mth]
postpaid_df = postpaid_df.loc[postpaid_df['PRODUCT']==product]
# postpaid_df = postpaid_df.loc[postpaid_df['CHANNEL_GRP']==channel_grp]

postpaid_df['ALL'] = postpaid_df['P']
postpaid_df['TRUE'] = np.where(postpaid_df['COMP_CD']=='TRUE', postpaid_df['P'], 0)
postpaid_df['DTAC'] = np.where(postpaid_df['COMP_CD']=='DTAC', postpaid_df['P'], 0)
postpaid_df['TMP_NAME'] = postpaid_df['METRIC_NAME'].replace(r' : DTAC| : TMH', '', regex=True)

postpaid_df = postpaid_df.groupby(['PRODUCT', 'METRIC_GRP', 'TMP_NAME']).agg({'LOAD_DATE':'max', 'START_DAY':'min', 'END_DAY':'max', 'ALL':'sum', 'TRUE':'sum', 'DTAC':'sum'})
postpaid_df = postpaid_df.fillna(0).sort_values(by=['PRODUCT', 'METRIC_GRP', 'TMP_NAME']).reset_index()

mod_col_list = postpaid_df.iloc[:, 6:9].columns.tolist()
for col in mod_col_list:
    postpaid_df[col] = postpaid_df[col].apply(lambda x: format(x, ',.0f'))

postpaid_df

,PRODUCT,METRIC_GRP,TMP_NAME,LOAD_DATE,START_DAY,END_DAY,ALL,TRUE,DTAC
0,POSTPAID,Gross Adds,Postpaid Gross Adds B2B,2025-06-20 08:47:41,20250601,20250619,"21,958","16,371","5,587"
1,POSTPAID,Gross Adds,Postpaid Gross Adds B2B - GEO Channel,2025-06-20 08:47:41,20250601,20250619,"2,535","1,974",561
2,POSTPAID,Gross Adds,Postpaid Gross Adds B2C,2025-06-20 08:47:41,20250601,20250619,"111,829","84,556","27,273"
3,POSTPAID,Gross Adds,Postpaid Gross Adds B2C - GEO Channel,2025-06-20 08:47:41,20250601,20250619,"45,124","37,476","7,648"
4,POSTPAID,Inflow M1,Postpaid Inflow M1 B2B,2025-06-20 08:47:41,20250601,20250619,"6,539,085","2,935,824","3,603,261"
5,POSTPAID,Inflow M1,Postpaid Inflow M1 B2B - GEO Channel,2025-06-20 08:47:41,20250601,20250619,"1,534,969","1,247,872","287,097"
6,POSTPAID,Inflow M1,Postpaid Inflow M1 B2C,2025-06-20 08:47:41,20250601,20250619,"47,710,708","36,480,710","11,229,998"
7,POSTPAID,Inflow M1,Postpaid Inflow M1 B2C - GEO Channel,2025-06-20 08:47:41,20250601,20250619,"21,561,477","17,219,623","4,341,854"


In [7]:
''' TOL Summary : 20 KPIs'''

curr_mth = chk_src_df['TM_KEY_MTH'].max()
product = 'TOL'
# channel_grp = 'GEO'

tol_df = chk_src_df.copy()
tol_df = tol_df.loc[tol_df['TM_KEY_MTH']==curr_mth]
tol_df = tol_df.loc[tol_df['PRODUCT']==product]
# tol_df = tol_df.loc[tol_df['CHANNEL_GRP']==channel_grp]

tol_df = tol_df.groupby(['PRODUCT', 'METRIC_GRP', 'METRIC_NAME']).agg({'LOAD_DATE':'max', 'START_DAY':'min', 'END_DAY':'max', 'P':'sum'})
tol_df = tol_df.fillna(0).sort_values(by=['PRODUCT', 'METRIC_GRP', 'METRIC_NAME']).reset_index()

mod_col_list = tol_df.iloc[:, 6:7].columns.tolist()
for col in mod_col_list:
    tol_df[col] = tol_df[col].apply(lambda x: format(x, ',.0f'))

tol_df

,PRODUCT,METRIC_GRP,METRIC_NAME,LOAD_DATE,START_DAY,END_DAY,P
0,TOL,Gross Adds,TOL Gross Adds Connected : Consumer,2025-06-20 08:00:10,20250601,20250619,"33,229"
1,TOL,Gross Adds,TOL Gross Adds Connected : Consumer (Install L...,2025-06-20 08:00:10,20250601,20250619,"33,229"
2,TOL,Gross Adds,TOL Gross Adds Connected : Consumer - GEO Channel,2025-06-20 08:00:10,20250601,20250619,"25,968"
3,TOL,Gross Adds,TOL Gross Adds Connected : Consumer - GEO Chan...,2025-06-20 08:00:10,20250601,20250619,"25,968"
4,TOL,Gross Adds,TOL Gross Adds Connected : DataService,2025-06-20 08:00:10,20250601,20250619,"1,005"
5,TOL,Gross Adds,TOL Gross Adds Connected : DataService (Instal...,2025-06-20 08:00:10,20250601,20250619,"1,005"
6,TOL,Gross Adds,TOL Gross Adds Connected : DataService - GEO C...,2025-06-20 08:00:10,20250602,20250617,23
7,TOL,Gross Adds,TOL Gross Adds Connected : DataService - GEO C...,2025-06-20 08:00:10,20250602,20250617,23
8,TOL,Gross Adds,TOL Gross Adds Registration : Consumer,2025-06-20 08:00:10,20250601,20250619,"46,487"
9,TOL,Gross Adds,TOL Gross Adds Registration : Consumer - GEO C...,2025-06-20 08:00:10,20250601,20250619,"35,733"


In [8]:
''' TVS Summary : 8 KPIs '''

curr_mth = chk_src_df['TM_KEY_MTH'].max()
product = 'TVS'
# channel_grp = 'GEO'

tvs_df = chk_src_df.copy()
tvs_df = tvs_df.loc[tvs_df['TM_KEY_MTH']==curr_mth]
tvs_df = tvs_df.loc[tvs_df['PRODUCT']==product]
# tvs_df = tvs_df.loc[tvs_df['CHANNEL_GRP']==channel_grp]

tvs_df = tvs_df.groupby(['PRODUCT', 'METRIC_GRP', 'METRIC_NAME']).agg({'LOAD_DATE':'max', 'START_DAY':'min', 'END_DAY':'max', 'P':'sum'})
tvs_df = tvs_df.fillna(0).sort_values(by=['PRODUCT', 'METRIC_GRP', 'METRIC_NAME']).reset_index()

mod_col_list = tvs_df.iloc[:, 6:7].columns.tolist()
for col in mod_col_list:
    tvs_df[col] = tvs_df[col].apply(lambda x: format(x, ',.0f'))

tvs_df

,PRODUCT,METRIC_GRP,METRIC_NAME,LOAD_DATE,START_DAY,END_DAY,P
0,TVS,Gross Adds,TVS CMDU Gross Adds,2025-06-20 08:49:26,20250601,20250613,30
1,TVS,Gross Adds,TVS Now Gross Adds,2025-06-20 08:49:26,20250601,20250618,"1,165"
2,TVS,Gross Adds,TVS Now Gross Adds - GEO Channel,2025-06-20 08:49:26,20250601,20250618,"1,161"
3,TVS,Inflow M1,TVS CMDU Inflow M1,2025-06-20 08:49:26,20250601,20250613,"173,248"
4,TVS,Inflow M1,TVS Now Inflow M1,2025-06-20 08:49:26,20250601,20250618,"275,592"
5,TVS,Inflow M1,TVS Now Inflow M1 - GEO Channel,2025-06-20 08:49:26,20250601,20250618,"275,147"


### Monitoring

In [9]:
''' Current Month Gross Adds '''

curr_mth = chk_src_df['TM_KEY_MTH'].max()
ga_metric_list = [
    'DB2S010100CS' #Postpaid Gross Adds B2C : DTAC
    , 'DB2S020100CS' #Postpaid Gross Adds B2B : DTAC
    , 'TB2S010100CS' #Postpaid Gross Adds B2C : TMH
    , 'TB2S020100CS' #Postpaid Gross Adds B2B : TMH
    , 'DB1S000101CS' #Prepaid Gross Adds : DTAC
    , 'TB1S000101CS' #Prepaid Gross Adds : TMH
    , 'TB3S000102CS' #TOL Gross Adds Connected : Consumer
    , 'TB3S000103CS' #TOL Gross Adds Connected : DataService
    , 'TB3S000104CS' #TOL Gross Adds Registration : Consumer
    , 'TB3S000105CS' #TOL Gross Adds Registration : DataService
    , 'TB4S001004CS' #TVS CMDU Gross Adds
    , 'TB4S001400CS' #TVS Now Gross Adds
    ]

src_ga_overall_df = chk_src_df.copy()
src_ga_overall_df = src_ga_overall_df.loc[src_ga_overall_df['TM_KEY_MTH']==curr_mth]
src_ga_overall_df = src_ga_overall_df.loc[src_ga_overall_df['METRIC_CD'].isin(ga_metric_list)]
src_ga_overall_df = src_ga_overall_df.groupby(['PRODUCT', 'METRIC_CD', 'METRIC_NAME']).agg({'LOAD_DATE':'max', 'START_DAY':'min', 'END_DAY':'max', 'C':'sum', 'P':'sum', 'G':'sum', 'H':'sum', 'HH':'sum', 'CCAA':'sum', 'CCAATT':'sum', 'CNT_AREA_TYPE':'max', 'CNT_AREA_CD':'max', 'ROW_CNT':'sum'})
src_ga_overall_df = src_ga_overall_df.fillna(0).sort_values(by=['PRODUCT', 'METRIC_CD']).reset_index()

mod_col_list = src_ga_overall_df.iloc[:, 6:13].columns.tolist()
for col in mod_col_list:
    src_ga_overall_df[col] = src_ga_overall_df[col].apply(lambda x: format(x, ',.0f'))

src_ga_overall_df

,PRODUCT,METRIC_CD,METRIC_NAME,LOAD_DATE,START_DAY,END_DAY,C,P,G,H,HH,CCAA,CCAATT,CNT_AREA_TYPE,CNT_AREA_CD,ROW_CNT
0,POSTPAID,DB2S010100CS,Postpaid Gross Adds B2C : DTAC,2025-06-20 08:47:41,20250601,20250618,0,"27,273","27,091","27,091","27,091","27,091","27,091",7,5265,25713
1,POSTPAID,DB2S020100CS,Postpaid Gross Adds B2B : DTAC,2025-06-20 08:47:41,20250601,20250618,0,"5,587","1,919","1,919","1,919","1,919","1,919",7,743,2467
2,POSTPAID,TB2S010100CS,Postpaid Gross Adds B2C : TMH,2025-06-20 08:47:41,20250601,20250619,0,"84,556","54,869","54,869","54,869","54,869","54,869",7,1333,17060
3,POSTPAID,TB2S020100CS,Postpaid Gross Adds B2B : TMH,2025-06-20 08:47:41,20250601,20250619,0,"16,371","1,977","1,977","1,977","1,977","1,977",7,606,2871
4,PREPAID,DB1S000101CS,Prepaid Gross Adds : DTAC,2025-06-20 08:02:16,20250601,20250618,0,"553,564","539,739","539,739","527,210","539,739","536,115",7,6829,64940
5,PREPAID,TB1S000101CS,Prepaid Gross Adds : TMH,2025-06-20 08:02:16,20250601,20250619,0,"480,014","365,110","365,110","355,521","365,110","365,073",7,5745,68197
6,TOL,TB3S000102CS,TOL Gross Adds Connected : Consumer,2025-06-20 08:00:10,20250601,20250619,0,"33,229","25,968","25,968","25,690","25,968","25,967",7,1137,14264
7,TOL,TB3S000103CS,TOL Gross Adds Connected : DataService,2025-06-20 08:00:10,20250601,20250619,0,"1,005",23,23,23,23,23,7,57,97
8,TOL,TB3S000104CS,TOL Gross Adds Registration : Consumer,2025-06-20 08:00:10,20250601,20250619,0,"46,487","35,727","35,727","35,350","35,727","35,726",7,1141,14697
9,TOL,TB3S000105CS,TOL Gross Adds Registration : DataService,2025-06-20 08:00:10,20250602,20250619,0,"1,188",33,33,33,33,33,7,66,109


In [10]:
''' Average Gross Adds per Month '''

avg_range_start = '202401'
avg_range_end = '202503'
ga_metric_list = [
    'DB2S010100CS' #Postpaid Gross Adds B2C : DTAC
    , 'DB2S020100CS' #Postpaid Gross Adds B2B : DTAC
    , 'TB2S010100CS' #Postpaid Gross Adds B2C : TMH
    , 'TB2S020100CS' #Postpaid Gross Adds B2B : TMH
    , 'DB1S000101CS' #Prepaid Gross Adds : DTAC
    , 'TB1S000101CS' #Prepaid Gross Adds : TMH
    , 'TB3S000102CS' #TOL Gross Adds Connected : Consumer
    , 'TB3S000103CS' #TOL Gross Adds Connected : DataService
    , 'TB3S000104CS' #TOL Gross Adds Registration : Consumer
    , 'TB3S000105CS' #TOL Gross Adds Registration : DataService
    , 'TB4S001004CS' #TVS CMDU Gross Adds
    , 'TB4S001400CS' #TVS Now Gross Adds
    ]

src_ga_overall_df = chk_src_df.copy()
src_ga_overall_df = src_ga_overall_df.loc[src_ga_overall_df['TM_KEY_MTH'].between(avg_range_start, avg_range_end)]
src_ga_overall_df = src_ga_overall_df.loc[src_ga_overall_df['METRIC_CD'].isin(ga_metric_list)]
src_ga_overall_df = src_ga_overall_df.groupby(['PRODUCT', 'METRIC_CD', 'METRIC_NAME']).agg({'LOAD_DATE':'max', 'START_DAY':'min', 'END_DAY':'max', 'C':'mean', 'P':'mean', 'G':'mean', 'H':'mean', 'HH':'mean', 'CCAA':'mean', 'CCAATT':'mean', 'CNT_AREA_TYPE':'max', 'CNT_AREA_CD':'max', 'ROW_CNT':'sum'})
src_ga_overall_df = src_ga_overall_df.fillna(0).sort_values(by=['PRODUCT', 'METRIC_CD']).reset_index()

mod_col_list = src_ga_overall_df.iloc[:, 6:13].columns.tolist()
for col in mod_col_list:
    src_ga_overall_df[col] = src_ga_overall_df[col].apply(lambda x: format(x, ',.0f'))

src_ga_overall_df

,PRODUCT,METRIC_CD,METRIC_NAME,LOAD_DATE,START_DAY,END_DAY,C,P,G,H,HH,CCAA,CCAATT,CNT_AREA_TYPE,CNT_AREA_CD,ROW_CNT
0,POSTPAID,DB2S010100CS,Postpaid Gross Adds B2C : DTAC,2025-06-07 14:21:18,20240101,20250331,0,"61,560","59,720","59,720","59,720","59,720","59,720",7,6204,697820
1,POSTPAID,DB2S020100CS,Postpaid Gross Adds B2B : DTAC,2025-06-07 14:21:18,20240101,20250331,0,"10,050","5,366","5,366","5,366","5,366","5,366",7,1528,80500
2,POSTPAID,TB2S010100CS,Postpaid Gross Adds B2C : TMH,2025-06-07 14:21:18,20240101,20250331,0,"152,588","103,167","103,167","103,167","103,167","103,167",7,2211,438943
3,POSTPAID,TB2S020100CS,Postpaid Gross Adds B2B : TMH,2025-06-07 14:21:18,20240101,20250331,0,"25,021","8,124","8,124","8,124","8,124","8,124",7,781,64761
4,PREPAID,DB1S000101CS,Prepaid Gross Adds : DTAC,2025-06-18 17:45:57,20240101,20250331,0,"1,347,209","1,301,411","1,301,411","1,283,629","1,301,577","1,293,678",7,7441,1684530
5,PREPAID,TB1S000101CS,Prepaid Gross Adds : TMH,2025-06-18 17:45:57,20240101,20250331,0,"1,138,166","940,606","940,606","918,524","940,606","940,541",7,5831,1789493
6,TOL,TB3S000102CS,TOL Gross Adds Connected : Consumer,2025-04-30 14:02:03,20240102,20250331,0,"46,948","33,562","33,562","34,737","35,069","35,055",7,1214,331274
7,TOL,TB3S000103CS,TOL Gross Adds Connected : DataService,2025-04-30 14:02:03,20240102,20250331,0,"1,584",34,34,35,35,35,7,99,2156
8,TOL,TB3S000104CS,TOL Gross Adds Registration : Consumer,2025-04-30 14:02:02,20240101,20250331,0,"64,322","45,578","45,578","47,202","47,631","47,608",7,1243,348927
9,TOL,TB3S000105CS,TOL Gross Adds Registration : DataService,2025-04-30 14:02:02,20240101,20250331,0,"3,354","1,033","1,033","1,025","1,043","1,043",7,620,25738


In [11]:
''' Current Month Inflow '''

curr_mth = chk_src_df['TM_KEY_MTH'].max()
inflow_metric_list = [
    'DB2R010500CS' #Postpaid Inflow M1 B2C : DTAC
    , 'DB2R020500CS' #Postpaid Inflow M1 B2B : DTAC
    , 'TB2R010500CS' #Postpaid Inflow M1 B2C : TMH
    , 'TB2R020500CS' #Postpaid Inflow M1 B2B : TMH
    , 'DB1R000900CS' #Prepaid Inflow M1 : DTAC
    , 'TB1R000900CS' #Prepaid Inflow M1 : TMH
    , 'TB3R000601CS' #TOL Inflow M1 Connected : Consumer
    , 'TB3R000602CS' #TOL Inflow M1 Connected : DataService
    , 'TB4R001004CS' #TVS CMDU Inflow M1
    , 'TB4R001700CS' #TVS Now Inflow M1
    , 'DB1R001000CS' #Prepaid Inflow M2 : DTAC
    , 'TB1R001000CS' #Prepaid Inflow M2 : TMH
    ]

src_inflow_overall_df = chk_src_df.copy()
src_inflow_overall_df = src_inflow_overall_df.loc[src_inflow_overall_df['TM_KEY_MTH']==curr_mth]
src_inflow_overall_df = src_inflow_overall_df.loc[src_inflow_overall_df['METRIC_CD'].isin(inflow_metric_list)]
src_inflow_overall_df = src_inflow_overall_df.groupby(['PRODUCT', 'METRIC_CD', 'METRIC_NAME']).agg({'LOAD_DATE':'max', 'START_DAY':'min', 'END_DAY':'max', 'C':'sum', 'P':'sum', 'G':'sum', 'H':'sum', 'HH':'sum', 'CCAA':'sum', 'CCAATT':'sum', 'CNT_AREA_TYPE':'max', 'CNT_AREA_CD':'max', 'ROW_CNT':'sum'})
src_inflow_overall_df = src_inflow_overall_df.fillna(0).sort_values(by=['PRODUCT', 'METRIC_CD']).reset_index()

mod_col_list = src_inflow_overall_df.iloc[:, 6:13].columns.tolist()
for col in mod_col_list:
    src_inflow_overall_df[col] = src_inflow_overall_df[col].apply(lambda x: format(x, ',.0f'))

src_inflow_overall_df

,PRODUCT,METRIC_CD,METRIC_NAME,LOAD_DATE,START_DAY,END_DAY,C,P,G,H,HH,CCAA,CCAATT,CNT_AREA_TYPE,CNT_AREA_CD,ROW_CNT
0,POSTPAID,DB2R010500CS,Postpaid Inflow M1 B2C : DTAC,2025-06-20 08:47:41,20250601,20250618,0,"11,229,998","11,130,363","11,130,363","11,130,363","11,130,363","11,130,363",7,5265,25713
1,POSTPAID,DB2R020500CS,Postpaid Inflow M1 B2B : DTAC,2025-06-20 08:47:41,20250601,20250618,0,"3,603,261","693,107","693,107","693,107","693,107","693,107",7,743,2467
2,POSTPAID,TB2R010500CS,Postpaid Inflow M1 B2C : TMH,2025-06-20 08:47:41,20250601,20250619,0,"36,480,710","27,506,772","27,506,772","27,506,772","27,506,772","27,506,772",7,1333,17060
3,POSTPAID,TB2R020500CS,Postpaid Inflow M1 B2B : TMH,2025-06-20 08:47:41,20250601,20250619,0,"2,935,824","1,249,220","1,249,220","1,249,220","1,249,220","1,249,220",7,606,2871
4,PREPAID,DB1R000900CS,Prepaid Inflow M1 : DTAC,2025-06-20 08:02:16,20250601,20250618,0,"111,436,431","109,726,663","109,726,663","106,827,057","109,726,663","108,878,162",7,6824,89406
5,PREPAID,DB1R001000CS,Prepaid Inflow M2 : DTAC,2025-06-20 08:02:16,20250601,20250618,0,"45,240,894","44,859,809","44,859,809","44,370,064","44,859,809","44,380,868",7,6933,115325
6,PREPAID,TB1R000900CS,Prepaid Inflow M1 : TMH,2025-06-20 08:02:16,20250601,20250617,0,"89,615,029","64,232,184","64,232,184","62,407,531","64,232,184","64,226,710",7,5664,72386
7,PREPAID,TB1R001000CS,Prepaid Inflow M2 : TMH,2025-06-20 08:02:16,20250601,20250617,0,"37,457,564","36,758,268","36,758,268","36,214,789","36,758,268","36,755,563",7,5714,85422
8,TOL,TB3R000601CS,TOL Inflow M1 Connected : Consumer,2025-06-20 08:00:10,20250601,20250619,0,"17,881,893","14,116,158","14,116,158","13,958,406","14,116,158","14,115,659",7,1137,14264
9,TOL,TB3R000602CS,TOL Inflow M1 Connected : DataService,2025-06-20 08:00:10,20250601,20250619,0,"859,541","58,677","58,677","58,677","58,677","58,677",7,57,97


In [12]:
''' Average Inflow per Month '''

avg_range_start = '202401'
avg_range_end = '202503'
inflow_metric_list = [
    'DB2R010500CS' #Postpaid Inflow M1 B2C : DTAC
    , 'DB2R020500CS' #Postpaid Inflow M1 B2B : DTAC
    , 'TB2R010500CS' #Postpaid Inflow M1 B2C : TMH
    , 'TB2R020500CS' #Postpaid Inflow M1 B2B : TMH
    , 'DB1R000900CS' #Prepaid Inflow M1 : DTAC
    , 'TB1R000900CS' #Prepaid Inflow M1 : TMH
    , 'TB3R000601CS' #TOL Inflow M1 Connected : Consumer
    , 'TB3R000602CS' #TOL Inflow M1 Connected : DataService
    , 'TB4R001004CS' #TVS CMDU Inflow M1
    , 'TB4R001700CS' #TVS Now Inflow M1
    , 'DB1R001000CS' #Prepaid Inflow M2 : DTAC
    , 'TB1R001000CS' #Prepaid Inflow M2 : TMH
    ]

src_inflow_overall_df = chk_src_df.copy()
src_inflow_overall_df = src_inflow_overall_df.loc[src_inflow_overall_df['TM_KEY_MTH'].between(avg_range_start, avg_range_end)]
src_inflow_overall_df = src_inflow_overall_df.loc[src_inflow_overall_df['METRIC_CD'].isin(inflow_metric_list)]
src_inflow_overall_df = src_inflow_overall_df.groupby(['PRODUCT', 'METRIC_CD', 'METRIC_NAME']).agg({'LOAD_DATE':'max', 'START_DAY':'min', 'END_DAY':'max', 'C':'sum', 'P':'sum', 'G':'sum', 'H':'sum', 'HH':'sum', 'CCAA':'sum', 'CCAATT':'sum', 'CNT_AREA_TYPE':'max', 'CNT_AREA_CD':'max', 'ROW_CNT':'sum'})
src_inflow_overall_df = src_inflow_overall_df.fillna(0).sort_values(by=['PRODUCT', 'METRIC_CD']).reset_index()

mod_col_list = src_inflow_overall_df.iloc[:, 6:13].columns.tolist()
for col in mod_col_list:
    src_inflow_overall_df[col] = src_inflow_overall_df[col].apply(lambda x: format(x, ',.0f'))

src_inflow_overall_df

,PRODUCT,METRIC_CD,METRIC_NAME,LOAD_DATE,START_DAY,END_DAY,C,P,G,H,HH,CCAA,CCAATT,CNT_AREA_TYPE,CNT_AREA_CD,ROW_CNT
0,POSTPAID,DB2R010500CS,Postpaid Inflow M1 B2C : DTAC,2025-06-07 14:21:18,20240101,20250331,0,"399,214,352","390,238,209","390,238,209","390,238,209","390,238,209","390,238,209",7,6204,697820
1,POSTPAID,DB2R020500CS,Postpaid Inflow M1 B2B : DTAC,2025-06-07 14:21:18,20240101,20250331,0,"146,087,502","27,656,846","27,656,846","27,656,846","27,656,846","27,656,846",7,1528,80500
2,POSTPAID,TB2R010500CS,Postpaid Inflow M1 B2C : TMH,2025-06-07 14:21:18,20240101,20250331,0,"963,708,492","742,500,893","742,500,893","742,500,893","742,500,893","742,500,893",7,2211,438943
3,POSTPAID,TB2R020500CS,Postpaid Inflow M1 B2B : TMH,2025-06-07 14:21:18,20240101,20250331,0,"84,587,995","36,558,027","36,558,027","36,558,027","36,558,027","36,558,027",7,781,64761
4,PREPAID,DB1R000900CS,Prepaid Inflow M1 : DTAC,2025-06-18 17:45:56,20240101,20250331,0,"3,802,215,253","3,738,940,932","3,738,940,932","3,680,837,448","3,739,497,664","3,714,680,253",7,7418,2535983
5,PREPAID,DB1R001000CS,Prepaid Inflow M2 : DTAC,2025-06-18 17:45:56,20240201,20250331,0,"1,083,316,091","1,068,960,271","1,068,960,271","1,056,615,717","1,069,126,998","1,056,950,051",7,7294,2747494
6,PREPAID,TB1R000900CS,Prepaid Inflow M1 : TMH,2025-06-18 17:45:56,20240101,20250331,0,"2,938,521,504","2,535,601,009","2,535,601,009","2,461,916,507","2,535,601,009","2,535,437,768",7,5800,2096987
7,PREPAID,TB1R001000CS,Prepaid Inflow M2 : TMH,2025-06-18 17:45:56,20240101,20250331,0,"1,060,137,134","1,049,549,983","1,049,549,983","1,030,456,436","1,049,549,983","1,049,435,343",7,5739,2269687
8,TOL,TB3R000601CS,TOL Inflow M1 Connected : Consumer,2025-04-30 14:02:02,20240102,20250331,0,"388,750,845","283,220,937","283,220,937","293,275,723","296,337,595","296,214,970",7,1214,331274
9,TOL,TB3R000602CS,TOL Inflow M1 Connected : DataService,2025-04-30 14:02:02,20240102,20250331,0,"29,369,420","1,591,398","1,591,398","1,627,784","1,630,883","1,630,883",7,99,2156


### All Metric

In [13]:
''' METRIC Summary '''

# src_t_sales_mth_df = chk_src_df.copy()
# src_t_sales_mth_df = src_t_sales_mth_df.groupby(['PRODUCT', 'METRIC_CD', 'METRIC_NAME']).agg({'LOAD_DATE':'max', 'START_DAY':'min', 'END_DAY':'max', 'P':'sum', 'CNT_AREA_TYPE':'max', 'CNT_AREA_CD':'max', 'ROW_CNT':'sum'})
# src_t_sales_mth_df = src_t_sales_mth_df.fillna(0).sort_values(by=['PRODUCT', 'METRIC_CD']).reset_index()

# mod_col_list = src_t_sales_mth_df.iloc[:, 6:7].columns.tolist()
# for col in mod_col_list:
#     src_t_sales_mth_df[col] = src_t_sales_mth_df[col].apply(lambda x: format(x, ',.0f'))

# print(f'\n{src_t_sales_mth_df.to_string(max_cols=10)}')

' METRIC Summary '

In [14]:
''' METRIC Current Month '''

# # v_tm_key_mth = '202502'
# curr_mth = chk_src_df['TM_KEY_MTH'].max()
# src_t_sales_mth_df_cols = ['PRODUCT', 'METRIC_CD', 'METRIC_NAME', 'LOAD_DATE', 'TM_KEY_MTH', 'START_DAY', 'END_DAY', 'C', 'P', 'G', 'H', 'HH', 'CCAA', 'CCAATT']
# print(f'\nParameter input...\n')
# print(f'   -> v_tm_key_mth: {curr_mth}')

# src_t_sales_mth_df = chk_src_df[src_t_sales_mth_df_cols].copy()
# src_t_sales_mth_df = src_t_sales_mth_df.loc[src_t_sales_mth_df['TM_KEY_MTH']==curr_mth]
# src_t_sales_mth_df = src_t_sales_mth_df.fillna(0).sort_values(by=['PRODUCT', 'METRIC_CD', 'TM_KEY_MTH']).reset_index(drop=True)

# mod_col_list = src_t_sales_mth_df.iloc[:, 7:14].columns.tolist()
# for col in mod_col_list:
#     src_t_sales_mth_df[col] = src_t_sales_mth_df[col].apply(lambda x: format(x, ',.0f'))

# print(f'\n{src_t_sales_mth_df.to_string(max_cols=17)}')

' METRIC Current Month '

## Actual : Transaction
->  DIM_CORP_KPI

In [15]:
''' Execute Daily by case '''


# Input parameter
curr_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
v_start_date = 20250101
print(f'\nParameter input...\n')
print(f'   -> v_start_date: {v_start_date}')


# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
print(f'\n{TDMDBPR_db} : Connected')
src_cur = src_conn.cursor()
query = (f"""
    SELECT /*+PARALLEL(8)*/
        PRODUCT, METRIC_CD, METRIC_NAME, TM_KEY_DAY
        , MAX(LOAD_DATE) LOAD_DATE
        , SUM(CASE WHEN AREA_TYPE = 'C' THEN METRIC_VALUE END) C
        , SUM(CASE WHEN AREA_TYPE = 'P' THEN METRIC_VALUE END) P
        , SUM(CASE WHEN AREA_TYPE = 'G' THEN METRIC_VALUE END) G
        , SUM(CASE WHEN AREA_TYPE = 'H' THEN METRIC_VALUE END) H
        , SUM(CASE WHEN AREA_TYPE = 'HH' THEN METRIC_VALUE END) HH
        , SUM(CASE WHEN AREA_TYPE = 'CCAA' THEN METRIC_VALUE END) CCAA
        , SUM(CASE WHEN AREA_TYPE = 'CCAATT' THEN METRIC_VALUE END) CCAATT
    FROM CDSAPPO.DIM_CORP_KPI
    WHERE REGEXP_LIKE(METRIC_CD, 'CS$|CG$') -->> 2025 KPIs
    AND EXISTS (
        SELECT 1 -->> AREA_CD IN (P, G, H, R, HH, CCAA, CCAATT)
        FROM (
            SELECT DISTINCT ORGID_P AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT ORGID_G AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT ORGID_R AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT ORGID_H AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT ORGID_HH AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT SUBSTR(CCAATT,1,4) AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT CCAATT AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
        ) TMP
        WHERE TMP_CD = AREA_CD )
    AND TM_KEY_DAY >= {v_start_date}
    GROUP BY PRODUCT, METRIC_CD, METRIC_NAME, TM_KEY_DAY
""")


try:
    execute_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
    print(f'\n   -> Execute query... {execute_datetime}')

    # Get : Summary DataFrame
    src_cur.execute(query)
    rows = src_cur.fetchall()
    chk_src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])
    print(f'\n   -> Daily DataFrame : {chk_src_df.shape[0]} rows, {chk_src_df.shape[1]} columns')

    src_cur.close()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()
    print(f'\n{TDMDBPR_db} : Disconnected')


Parameter input...

   -> v_start_date: 20250101

TDMDBPR : Connected

   -> Execute query... 2025-06-20, 10:51:01

   -> Daily DataFrame : 11394 rows, 12 columns

TDMDBPR : Disconnected


In [16]:
''' METRIC by Day '''

v_tm_key_day = 20250101
print(f'\nParameter input...\n')
print(f'   -> v_tm_key_day: {v_tm_key_day}')

src_sale_day_df = chk_src_df.copy()
src_sale_day_df = src_sale_day_df.loc[src_sale_day_df['TM_KEY_DAY']>=v_tm_key_day]
src_sale_day_df = src_sale_day_df.fillna(0).sort_values(by=['METRIC_CD', 'TM_KEY_DAY']).reset_index(drop=True)

mod_col_list = src_sale_day_df.iloc[:, 5:12].columns.tolist()
for col in mod_col_list:
    src_sale_day_df[col] = src_sale_day_df[col].apply(lambda x: format(x, ',.0f'))

print(f'\nCreate : src_sale_day_df...')
print(f'\n   -> DataFrame : {chk_src_df.shape[0]} rows, {chk_src_df.shape[1]} columns')
# print(f'\n{src_sale_day_df.to_string(max_cols=12)}')


Parameter input...

   -> v_tm_key_day: 20250101

Create : src_sale_day_df...

   -> DataFrame : 11394 rows, 12 columns


In [17]:
# ''' xB1S000101CS : Prepaid Gross Adds : xxx '''

# v_tm_key_day = 20250401
# v_metric_cd = ['DB1S000101CS', 'TB1S000101CS']
# print(f'\nDB1S000101CS : Prepaid Gross Adds : DTAC\nTB1S000101CS : Prepaid Gross Adds : TMH\n')
# print(f'   -> v_metric_cd: {v_metric_cd}')
# print(f'   -> v_tm_key_day: {v_tm_key_day}')

# xB1S000101CS_day_df = chk_src_df.copy()
# xB1S000101CS_day_df = xB1S000101CS_day_df.loc[xB1S000101CS_day_df['TM_KEY_DAY']>=v_tm_key_day]
# xB1S000101CS_day_df = xB1S000101CS_day_df.loc[xB1S000101CS_day_df['METRIC_CD'].isin(v_metric_cd)]
# xB1S000101CS_day_df = xB1S000101CS_day_df.fillna(0).sort_values(by=['PRODUCT', 'METRIC_CD', 'TM_KEY_DAY']).reset_index(drop=True)

# mod_col_list = xB1S000101CS_day_df.iloc[:, 5:12].columns.tolist()
# for col in mod_col_list:
#     xB1S000101CS_day_df[col] = xB1S000101CS_day_df[col].apply(lambda x: format(x, ',.0f'))
    
# # xB1S000101CS_day_df
# print(f'\n{xB1S000101CS_day_df.to_string(max_cols=12)}')

In [18]:
''' TB1S000101CS : Prepaid Gross Adds : TMH '''

v_tm_key_day = 20250401
v_metric_cd = 'TB1S000101CS'
print(f'\nTB1S000101CS : Prepaid Gross Adds : TMH\n')
print(f'   -> v_metric_cd: {v_metric_cd}')
print(f'   -> v_tm_key_day: {v_tm_key_day}')

TB1S000101CS_day_df = chk_src_df.copy()
TB1S000101CS_day_df = TB1S000101CS_day_df.loc[TB1S000101CS_day_df['TM_KEY_DAY']>=v_tm_key_day]
TB1S000101CS_day_df = TB1S000101CS_day_df.loc[TB1S000101CS_day_df['METRIC_CD']==v_metric_cd]
TB1S000101CS_day_df = TB1S000101CS_day_df.fillna(0).sort_values(by=['PRODUCT', 'METRIC_CD', 'TM_KEY_DAY']).reset_index(drop=True)

mod_col_list = TB1S000101CS_day_df.iloc[:, 5:12].columns.tolist()
for col in mod_col_list:
    TB1S000101CS_day_df[col] = TB1S000101CS_day_df[col].apply(lambda x: format(x, ',.0f'))
    
TB1S000101CS_day_df


TB1S000101CS : Prepaid Gross Adds : TMH

   -> v_metric_cd: TB1S000101CS
   -> v_tm_key_day: 20250401


,PRODUCT,METRIC_CD,METRIC_NAME,TM_KEY_DAY,LOAD_DATE,C,P,G,H,HH,CCAA,CCAATT
0,PREPAID,TB1S000101CS,Prepaid Gross Adds : TMH,20250401,2025-06-18 17:45:57,0,"31,770","25,572","25,572","24,883","25,572","25,572"
1,PREPAID,TB1S000101CS,Prepaid Gross Adds : TMH,20250402,2025-06-18 17:45:57,0,"33,051","25,896","25,896","25,142","25,896","25,891"
2,PREPAID,TB1S000101CS,Prepaid Gross Adds : TMH,20250403,2025-06-18 17:45:57,0,"33,578","25,569","25,569","24,908","25,569","25,567"
3,PREPAID,TB1S000101CS,Prepaid Gross Adds : TMH,20250404,2025-06-18 17:45:57,0,"34,354","26,790","26,790","26,147","26,790","26,789"
4,PREPAID,TB1S000101CS,Prepaid Gross Adds : TMH,20250405,2025-06-18 17:45:57,0,"31,312","24,931","24,931","24,353","24,931","24,929"
...,...,...,...,...,...,...,...,...,...,...,...,...
75,PREPAID,TB1S000101CS,Prepaid Gross Adds : TMH,20250615,2025-06-20 08:02:16,0,"22,808","17,407","17,407","16,951","17,407","17,404"
76,PREPAID,TB1S000101CS,Prepaid Gross Adds : TMH,20250616,2025-06-20 08:02:16,0,"25,339","19,686","19,686","19,206","19,686","19,681"
77,PREPAID,TB1S000101CS,Prepaid Gross Adds : TMH,20250617,2025-06-20 08:02:16,0,"24,211","19,165","19,165","18,673","19,165","19,163"
78,PREPAID,TB1S000101CS,Prepaid Gross Adds : TMH,20250618,2025-06-20 08:02:16,0,"25,688","19,748","19,748","19,331","19,748","19,746"


In [19]:
''' DB1S000101CS : Prepaid Gross Adds : DTAC '''

v_tm_key_day = 20250401
v_metric_cd = 'DB1S000101CS'
print(f'\nDB1S000101CS : Prepaid Gross Adds : DTAC\n')
print(f'   -> v_metric_cd: {v_metric_cd}')
print(f'   -> v_tm_key_day: {v_tm_key_day}')

DB1S000101CS_day_df = chk_src_df.copy()
DB1S000101CS_day_df = DB1S000101CS_day_df.loc[DB1S000101CS_day_df['TM_KEY_DAY']>=v_tm_key_day]
DB1S000101CS_day_df = DB1S000101CS_day_df.loc[DB1S000101CS_day_df['METRIC_CD']==v_metric_cd]
DB1S000101CS_day_df = DB1S000101CS_day_df.fillna(0).sort_values(by=['PRODUCT', 'METRIC_CD', 'TM_KEY_DAY']).reset_index(drop=True)

mod_col_list = DB1S000101CS_day_df.iloc[:, 5:12].columns.tolist()
for col in mod_col_list:
    DB1S000101CS_day_df[col] = DB1S000101CS_day_df[col].apply(lambda x: format(x, ',.0f'))
    
DB1S000101CS_day_df


DB1S000101CS : Prepaid Gross Adds : DTAC

   -> v_metric_cd: DB1S000101CS
   -> v_tm_key_day: 20250401


,PRODUCT,METRIC_CD,METRIC_NAME,TM_KEY_DAY,LOAD_DATE,C,P,G,H,HH,CCAA,CCAATT
0,PREPAID,DB1S000101CS,Prepaid Gross Adds : DTAC,20250401,2025-06-18 17:45:57,0,"30,296","29,397","29,397","29,023","29,397","29,197"
1,PREPAID,DB1S000101CS,Prepaid Gross Adds : DTAC,20250402,2025-06-18 17:45:57,0,"31,654","31,020","31,020","30,578","31,020","30,853"
2,PREPAID,DB1S000101CS,Prepaid Gross Adds : DTAC,20250403,2025-06-18 17:45:57,0,"32,983","31,829","31,829","31,366","31,829","31,696"
3,PREPAID,DB1S000101CS,Prepaid Gross Adds : DTAC,20250404,2025-06-18 17:45:57,0,"31,180","30,236","30,236","29,767","30,236","30,097"
4,PREPAID,DB1S000101CS,Prepaid Gross Adds : DTAC,20250405,2025-06-18 17:45:57,0,"29,816","28,568","28,568","28,139","28,568","28,406"
...,...,...,...,...,...,...,...,...,...,...,...,...
74,PREPAID,DB1S000101CS,Prepaid Gross Adds : DTAC,20250614,2025-06-20 08:02:16,0,"27,667","27,263","27,263","26,622","27,263","27,111"
75,PREPAID,DB1S000101CS,Prepaid Gross Adds : DTAC,20250615,2025-06-20 08:02:16,0,"28,207","27,774","27,774","27,370","27,774","27,407"
76,PREPAID,DB1S000101CS,Prepaid Gross Adds : DTAC,20250616,2025-06-20 08:02:16,0,"28,704","27,600","27,600","26,912","27,600","27,437"
77,PREPAID,DB1S000101CS,Prepaid Gross Adds : DTAC,20250617,2025-06-20 08:02:16,0,"28,004","27,219","27,219","26,780","27,219","27,061"


In [20]:
# ''' xB2S010100CS : Postpaid Gross Adds B2C : xxx '''

# v_tm_key_day = 20250401
# v_metric_cd = ['DB2S010100CS', 'TB2S010100CS']
# print(f'\nDB2S010100CS : Postpaid Gross Adds : DTAC\nTB2S010100CS : Postpaid Gross Adds : TMH\n')
# print(f'   -> v_metric_cd: {v_metric_cd}')
# print(f'   -> v_tm_key_day: {v_tm_key_day}')

# xB2S010100CS_day_df = chk_src_df.copy()
# xB2S010100CS_day_df = xB2S010100CS_day_df.loc[xB2S010100CS_day_df['TM_KEY_DAY']>=v_tm_key_day]
# xB2S010100CS_day_df = xB2S010100CS_day_df.loc[xB2S010100CS_day_df['METRIC_CD'].isin(v_metric_cd)]
# xB2S010100CS_day_df = xB2S010100CS_day_df.fillna(0).sort_values(by=['PRODUCT', 'METRIC_CD', 'TM_KEY_DAY']).reset_index(drop=True)

# mod_col_list = xB2S010100CS_day_df.iloc[:, 5:12].columns.tolist()
# for col in mod_col_list:
#     xB2S010100CS_day_df[col] = xB2S010100CS_day_df[col].apply(lambda x: format(x, ',.0f'))
    
# # xB2S010100CS_day_df
# print(f'\n{xB2S010100CS_day_df.to_string(max_cols=12)}')

In [21]:
''' TB2S010100CS : Postpaid Gross Adds B2C : TMH '''

v_tm_key_day = 20250401
v_metric_cd = 'TB2S010100CS'
print(f'\nTB2S010100CS : Postpaid Gross Adds B2C : TMH\n')
print(f'   -> v_metric_cd: {v_metric_cd}')
print(f'   -> v_tm_key_day: {v_tm_key_day}')

TB2S010100CS_day_df = chk_src_df.copy()
TB2S010100CS_day_df = TB2S010100CS_day_df.loc[TB2S010100CS_day_df['TM_KEY_DAY']>=v_tm_key_day]
TB2S010100CS_day_df = TB2S010100CS_day_df.loc[TB2S010100CS_day_df['METRIC_CD']==v_metric_cd]
TB2S010100CS_day_df = TB2S010100CS_day_df.fillna(0).sort_values(by=['PRODUCT', 'METRIC_CD', 'TM_KEY_DAY']).reset_index(drop=True)

mod_col_list = TB2S010100CS_day_df.iloc[:, 5:12].columns.tolist()
for col in mod_col_list:
    TB2S010100CS_day_df[col] = TB2S010100CS_day_df[col].apply(lambda x: format(x, ',.0f'))
    
TB2S010100CS_day_df


TB2S010100CS : Postpaid Gross Adds B2C : TMH

   -> v_metric_cd: TB2S010100CS
   -> v_tm_key_day: 20250401


,PRODUCT,METRIC_CD,METRIC_NAME,TM_KEY_DAY,LOAD_DATE,C,P,G,H,HH,CCAA,CCAATT
0,POSTPAID,TB2S010100CS,Postpaid Gross Adds B2C : TMH,20250401,2025-06-20 08:47:41,0,"5,935","4,519","4,519","4,519","4,519","4,519"
1,POSTPAID,TB2S010100CS,Postpaid Gross Adds B2C : TMH,20250402,2025-06-20 08:47:41,0,"5,268","3,746","3,746","3,746","3,746","3,746"
2,POSTPAID,TB2S010100CS,Postpaid Gross Adds B2C : TMH,20250403,2025-06-20 08:47:41,0,"5,044","3,485","3,485","3,485","3,485","3,485"
3,POSTPAID,TB2S010100CS,Postpaid Gross Adds B2C : TMH,20250404,2025-06-20 08:47:41,0,"5,261","3,575","3,575","3,575","3,575","3,575"
4,POSTPAID,TB2S010100CS,Postpaid Gross Adds B2C : TMH,20250405,2025-06-20 08:47:41,0,"5,445","3,882","3,882","3,882","3,882","3,882"
...,...,...,...,...,...,...,...,...,...,...,...,...
75,POSTPAID,TB2S010100CS,Postpaid Gross Adds B2C : TMH,20250615,2025-06-20 08:47:41,0,"4,279","3,133","3,133","3,133","3,133","3,133"
76,POSTPAID,TB2S010100CS,Postpaid Gross Adds B2C : TMH,20250616,2025-06-20 08:47:41,0,"3,741","2,310","2,310","2,310","2,310","2,310"
77,POSTPAID,TB2S010100CS,Postpaid Gross Adds B2C : TMH,20250617,2025-06-20 08:47:41,0,"4,428","2,995","2,995","2,995","2,995","2,995"
78,POSTPAID,TB2S010100CS,Postpaid Gross Adds B2C : TMH,20250618,2025-06-20 08:47:41,0,"3,589","2,321","2,321","2,321","2,321","2,321"


In [22]:
''' DB2S010100CS : Postpaid Gross Adds B2C : DTAC '''

v_tm_key_day = 20250401
v_metric_cd = 'DB2S010100CS'
print(f'\nDB2S010100CS : Postpaid Gross Adds B2C : DTAC\n')
print(f'   -> v_metric_cd: {v_metric_cd}')
print(f'   -> v_tm_key_day: {v_tm_key_day}')

DB2S010100CS_day_df = chk_src_df.copy()
DB2S010100CS_day_df = DB2S010100CS_day_df.loc[DB2S010100CS_day_df['TM_KEY_DAY']>=v_tm_key_day]
DB2S010100CS_day_df = DB2S010100CS_day_df.loc[DB2S010100CS_day_df['METRIC_CD']==v_metric_cd]
DB2S010100CS_day_df = DB2S010100CS_day_df.fillna(0).sort_values(by=['PRODUCT', 'METRIC_CD', 'TM_KEY_DAY']).reset_index(drop=True)

mod_col_list = DB2S010100CS_day_df.iloc[:, 5:12].columns.tolist()
for col in mod_col_list:
    DB2S010100CS_day_df[col] = DB2S010100CS_day_df[col].apply(lambda x: format(x, ',.0f'))
    
DB2S010100CS_day_df


DB2S010100CS : Postpaid Gross Adds B2C : DTAC

   -> v_metric_cd: DB2S010100CS
   -> v_tm_key_day: 20250401


,PRODUCT,METRIC_CD,METRIC_NAME,TM_KEY_DAY,LOAD_DATE,C,P,G,H,HH,CCAA,CCAATT
0,POSTPAID,DB2S010100CS,Postpaid Gross Adds B2C : DTAC,20250401,2025-06-20 08:47:41,0,"1,274","1,267","1,267","1,267","1,267","1,267"
1,POSTPAID,DB2S010100CS,Postpaid Gross Adds B2C : DTAC,20250402,2025-06-20 08:47:41,0,"1,395","1,386","1,386","1,386","1,386","1,386"
2,POSTPAID,DB2S010100CS,Postpaid Gross Adds B2C : DTAC,20250403,2025-06-20 08:47:41,0,"2,263","2,242","2,242","2,242","2,242","2,242"
3,POSTPAID,DB2S010100CS,Postpaid Gross Adds B2C : DTAC,20250404,2025-06-20 08:47:41,0,"2,199","2,170","2,170","2,170","2,170","2,170"
4,POSTPAID,DB2S010100CS,Postpaid Gross Adds B2C : DTAC,20250405,2025-06-20 08:47:41,0,"1,785","1,777","1,777","1,777","1,777","1,777"
...,...,...,...,...,...,...,...,...,...,...,...,...
74,POSTPAID,DB2S010100CS,Postpaid Gross Adds B2C : DTAC,20250614,2025-06-20 08:47:41,0,"1,174","1,169","1,169","1,169","1,169","1,169"
75,POSTPAID,DB2S010100CS,Postpaid Gross Adds B2C : DTAC,20250615,2025-06-20 08:47:41,0,724,716,716,716,716,716
76,POSTPAID,DB2S010100CS,Postpaid Gross Adds B2C : DTAC,20250616,2025-06-20 08:47:41,0,"1,876","1,869","1,869","1,869","1,869","1,869"
77,POSTPAID,DB2S010100CS,Postpaid Gross Adds B2C : DTAC,20250617,2025-06-20 08:47:41,0,"2,566","2,553","2,553","2,553","2,553","2,553"


In [23]:
''' TB3S000102CS : TOL Gross Adds Connected : Consumer '''

v_tm_key_day = 20250601
v_metric_cd = 'TB3S000102CS'
print(f'\nTB3S000102CS : TOL Gross Adds Connected : Consumer\n')
print(f'   -> v_metric_cd: {v_metric_cd}')
print(f'   -> v_tm_key_day: {v_tm_key_day}')

TB3S000102CS_day_df = chk_src_df.copy()
TB3S000102CS_day_df = TB3S000102CS_day_df.loc[TB3S000102CS_day_df['TM_KEY_DAY']>=v_tm_key_day]
TB3S000102CS_day_df = TB3S000102CS_day_df.loc[TB3S000102CS_day_df['METRIC_CD']==v_metric_cd]
TB3S000102CS_day_df = TB3S000102CS_day_df.fillna(0).sort_values(by=['PRODUCT', 'METRIC_CD', 'TM_KEY_DAY']).reset_index(drop=True)

mod_col_list = TB3S000102CS_day_df.iloc[:, 5:12].columns.tolist()
for col in mod_col_list:
    TB3S000102CS_day_df[col] = TB3S000102CS_day_df[col].apply(lambda x: format(x, ',.0f'))
    
TB3S000102CS_day_df


TB3S000102CS : TOL Gross Adds Connected : Consumer

   -> v_metric_cd: TB3S000102CS
   -> v_tm_key_day: 20250601


,PRODUCT,METRIC_CD,METRIC_NAME,TM_KEY_DAY,LOAD_DATE,C,P,G,H,HH,CCAA,CCAATT
0,TOL,TB3S000102CS,TOL Gross Adds Connected : Consumer,20250601,2025-06-20 08:00:10,0,"1,670","1,240","1,240","1,221","1,240","1,240"
1,TOL,TB3S000102CS,TOL Gross Adds Connected : Consumer,20250602,2025-06-20 08:00:10,0,"1,696","1,342","1,342","1,327","1,342","1,342"
2,TOL,TB3S000102CS,TOL Gross Adds Connected : Consumer,20250603,2025-06-20 08:00:10,0,"1,735","1,391","1,391","1,378","1,391","1,391"
3,TOL,TB3S000102CS,TOL Gross Adds Connected : Consumer,20250604,2025-06-20 08:00:10,0,"1,825","1,463","1,463","1,451","1,463","1,462"
4,TOL,TB3S000102CS,TOL Gross Adds Connected : Consumer,20250605,2025-06-20 08:00:10,0,"1,796","1,420","1,420","1,414","1,420","1,420"
5,TOL,TB3S000102CS,TOL Gross Adds Connected : Consumer,20250606,2025-06-20 08:00:10,0,"1,800","1,430","1,430","1,414","1,430","1,430"
6,TOL,TB3S000102CS,TOL Gross Adds Connected : Consumer,20250607,2025-06-20 08:00:10,0,"1,773","1,309","1,309","1,295","1,309","1,309"
7,TOL,TB3S000102CS,TOL Gross Adds Connected : Consumer,20250608,2025-06-20 08:00:10,0,"1,754","1,277","1,277","1,256","1,277","1,277"
8,TOL,TB3S000102CS,TOL Gross Adds Connected : Consumer,20250609,2025-06-20 08:00:10,0,"1,788","1,416","1,416","1,399","1,416","1,416"
9,TOL,TB3S000102CS,TOL Gross Adds Connected : Consumer,20250610,2025-06-20 08:00:10,0,"1,785","1,439","1,439","1,426","1,439","1,439"


In [24]:
''' TB4S001400CS : TVS Now Gross Adds '''

v_tm_key_day = 20250401
v_metric_cd = 'TB4S001400CS'
print(f'\nTB4S001400CS : TVS Now Gross Adds\n')
print(f'   -> v_metric_cd: {v_metric_cd}')
print(f'   -> v_tm_key_day: {v_tm_key_day}')

TB4S001400CS_day_df = chk_src_df.copy()
TB4S001400CS_day_df = TB4S001400CS_day_df.loc[TB4S001400CS_day_df['TM_KEY_DAY']>=v_tm_key_day]
TB4S001400CS_day_df = TB4S001400CS_day_df.loc[TB4S001400CS_day_df['METRIC_CD']==v_metric_cd]
TB4S001400CS_day_df = TB4S001400CS_day_df.fillna(0).sort_values(by=['PRODUCT', 'METRIC_CD', 'TM_KEY_DAY']).reset_index(drop=True)

mod_col_list = TB4S001400CS_day_df.iloc[:, 5:12].columns.tolist()
for col in mod_col_list:
    TB4S001400CS_day_df[col] = TB4S001400CS_day_df[col].apply(lambda x: format(x, ',.0f'))
    
TB4S001400CS_day_df


TB4S001400CS : TVS Now Gross Adds

   -> v_metric_cd: TB4S001400CS
   -> v_tm_key_day: 20250401


,PRODUCT,METRIC_CD,METRIC_NAME,TM_KEY_DAY,LOAD_DATE,C,P,G,H,HH,CCAA,CCAATT
0,TVS,TB4S001400CS,TVS Now Gross Adds,20250401,2025-06-20 08:49:26,0,28,28,28,28,28,28
1,TVS,TB4S001400CS,TVS Now Gross Adds,20250402,2025-06-20 08:49:26,0,30,30,30,30,30,30
2,TVS,TB4S001400CS,TVS Now Gross Adds,20250403,2025-06-20 08:49:26,0,37,37,37,37,37,37
3,TVS,TB4S001400CS,TVS Now Gross Adds,20250404,2025-06-20 08:49:26,0,59,59,59,59,59,59
4,TVS,TB4S001400CS,TVS Now Gross Adds,20250405,2025-06-20 08:49:26,0,61,61,61,61,61,61
...,...,...,...,...,...,...,...,...,...,...,...,...
74,TVS,TB4S001400CS,TVS Now Gross Adds,20250614,2025-06-20 08:49:26,0,47,43,43,43,43,43
75,TVS,TB4S001400CS,TVS Now Gross Adds,20250615,2025-06-20 08:49:26,0,44,42,42,42,42,42
76,TVS,TB4S001400CS,TVS Now Gross Adds,20250616,2025-06-20 08:49:26,0,49,47,47,47,47,47
77,TVS,TB4S001400CS,TVS Now Gross Adds,20250617,2025-06-20 08:49:26,0,59,55,55,55,55,55


## Actual : Issue
->  DIM_CORP_KPI

In [25]:
# ''' Execute Issue by case '''


# # Input parameter
# curr_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
# # v_start_date = 20250101
# print(f'\nParameter input...\n')
# # print(f'   -> v_start_date: {v_start_date}')


# # Connect : TDMDBPR
# src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
# src_conn = oracledb.connect(src_dsn)
# print(f'\n{TDMDBPR_db} : Connected')
# src_cur = src_conn.cursor()
# query = (f"""
#     SELECT /*+PARALLEL(8)*/ T.TM_KEY_DAY AS TM_KEY_MTH
#         , T.METRIC_CD, T.METRIC_NAME, T.AREA_TYPE, T.AREA_CD, T.AREA_DESC
#         , T.METRIC_VALUE AS TARGET
#         , A.ACTUAL
#         , O.ORGID_P, O.ZONE_TYPE, O.ORGID_G, O.TDS_SGMD, O.ORGID_H, O.HOP_HINT, O.ORGID_HH, O.D_CLUSTER
        
#     FROM CDSAPPO.DIM_CORP_TARGET T

#     LEFT JOIN (
#         SELECT SUBSTR(TM_KEY_DAY,1,6) AS TM_KEY_MTH
#             , PRODUCT, METRIC_CD, METRIC_NAME, AREA_TYPE, AREA_CD, AREA_DESC
#             , SUM(METRIC_VALUE) AS ACTUAL, MAX(LOAD_DATE) LOAD_DATE
#         FROM CDSAPPO.DIM_CORP_KPI
#         WHERE METRIC_CD = 'TB3S000102CS' --TOL Gross Adds Connected : Consumer
#         AND AREA_TYPE = 'CCAA'
#         AND TM_KEY_DAY >= 20250101
#         GROUP BY SUBSTR(TM_KEY_DAY,1,6), PRODUCT, METRIC_CD, METRIC_NAME, AREA_TYPE, AREA_CD, AREA_DESC
#     ) A
#         ON A.METRIC_CD = T.METRIC_CD
#         AND A.AREA_CD = T.AREA_CD
#         AND A.TM_KEY_MTH = T.TM_KEY_DAY
        
#     LEFT JOIN (
#         SELECT DISTINCT ORGID_P, ZONE_TYPE, ORGID_G, TDS_SGMD, ORGID_H, HOP_HINT, ORGID_HH, D_CLUSTER
#             , SUBSTR(CCAATT,1,2) AS PROVINCE_CD, PROVINCE_ENG 
#             , SUBSTR(CCAATT,1,4) AS CCAA, DISTRICT_EN
#         FROM CDSAPPO.DIM_MOOC_AREA NOLOCK
#         WHERE TEAM_CODE <> 'ไม่ระบุ' AND REMARK <> 'Dummy'
#     ) O
#         ON O.CCAA = T.AREA_CD

#     WHERE T.METRIC_CD = 'TB3S000102CS'
#     AND T.AREA_TYPE = 'CCAA'
#     AND T.TM_KEY_DAY >= 202501

#     --ORDER BY T.TM_KEY_DAY, T.METRIC_CD, T.AREA_TYPE, O.ZONE_TYPE, O.ORGID_G, O.ORGID_H, O.ORGID_HH
# """)


# try:
#     execute_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
#     print(f'\n   -> Execute query... {execute_datetime}')

#     # Get : Summary DataFrame
#     src_cur.execute(query)
#     rows = src_cur.fetchall()
#     chk_src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])
#     print(f'\n   -> Issue DataFrame : {chk_src_df.shape[0]} rows, {chk_src_df.shape[1]} columns')

#     src_cur.close()


# except oracledb.DatabaseError as e:
#     print(f'\nError with Oracle : {e}')


# finally:
#     src_conn.close()
#     print(f'\n{TDMDBPR_db} : Disconnected')

In [26]:
# ''' TB3S000102CS : TOL Gross Adds Connected : Consumer '''

# v_tm_key_mth = 202501
# # v_metric_cd = 'TB3S000102CS'
# print(f'\nTB3S000102CS : TOL Gross Adds Connected : Consumer\n')
# # print(f'   -> v_metric_cd: {v_metric_cd}')
# print(f'   -> v_tm_key_mth: {v_tm_key_mth}')

# TB3S000102CS_issue_df = chk_src_df.copy()
# TB3S000102CS_issue_df = TB3S000102CS_issue_df.loc[TB3S000102CS_issue_df['TM_KEY_MTH']==v_tm_key_mth]
# TB3S000102CS_issue_df = TB3S000102CS_issue_df.loc[TB3S000102CS_issue_df['ACTUAL'] > 0]
# # TB3S000102CS_issue_df = TB3S000102CS_issue_df.loc[TB3S000102CS_issue_df['ACTUAL'].isna()]
# # TB3S000102CS_issue_df = TB3S000102CS_issue_df.loc[TB3S000102CS_issue_df['METRIC_CD']==v_metric_cd]
# TB3S000102CS_issue_df = TB3S000102CS_issue_df.fillna(0).sort_values(by=['TM_KEY_MTH', 'METRIC_CD']).reset_index(drop=True)

# mod_col_list = TB3S000102CS_issue_df.iloc[:, 6:8].columns.tolist()
# for col in mod_col_list:
#     TB3S000102CS_issue_df[col] = TB3S000102CS_issue_df[col].apply(lambda x: format(x, ',.0f'))
    
# # TB3S000102CS_issue_df
# print(f'\n{TB3S000102CS_issue_df.to_string(max_cols=100)}')

## Output File
->  DIM_CORP_KPI

In [27]:
# ''' Execute Sales Monthly(Backup) '''


# # Input parameter
# curr_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
# v_start_date = 20240101
# print(f'\nParameter input...\n')
# print(f'   -> v_start_date: {v_start_date}')


# # Connect : TDMDBPR
# src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
# src_conn = oracledb.connect(src_dsn)
# print(f'\n{TDMDBPR_db} : Connected')
# src_cur = src_conn.cursor()
# query = (f"""
#     SELECT /*+PARALLEL(8)*/ 
#         TM_KEY_YR, TM_KEY_MTH
#         , 'Y2025 v.1' AS VER_FLAG
#         , PRODUCT_GRP AS PRODUCT, COMP_CD, METRIC_CD, METRIC_NAME
#         , NULL AS REMARK, NULL AS UPD_TM, NULL AS PPN_TM
#         , SUM(CASE WHEN AREA_TYPE = 'P' THEN ACTUAL_SNAP END) P_ACTUAL
#         , SUM(CASE WHEN AREA_TYPE = 'G' THEN ACTUAL_SNAP END) G
#         , SUM(CASE WHEN AREA_TYPE = 'H' THEN ACTUAL_SNAP END) H
#         , SUM(CASE WHEN AREA_TYPE = 'HH' THEN ACTUAL_SNAP END) HH
#         , SUM(CASE WHEN AREA_TYPE = 'CCAA' THEN ACTUAL_SNAP END) CCAA
#         , SUM(CASE WHEN AREA_TYPE = 'CCAATT' THEN ACTUAL_SNAP END) CCAATT
#         , SUM(CASE WHEN AREA_CD = '016' THEN ACTUAL_SNAP END) "016"
#         , SUM(CASE WHEN AREA_CD = '040' THEN ACTUAL_SNAP END) "040"
#         , SUM(CASE WHEN AREA_CD = '080' THEN ACTUAL_SNAP END) "080"
#         , MIN(TM_KEY_DAY) AS START_DAY, MAX(TM_KEY_DAY) AS END_DAY, MAX(PPN_TM) LOAD_DATE
#     FROM GEOSPCAPPO.AGG_PERF_NEWCO_CN_BK_2025_01_05
#     WHERE TM_KEY_DAY >= {v_start_date}
#     GROUP BY TM_KEY_YR, TM_KEY_MTH, PRODUCT_GRP, COMP_CD, METRIC_CD, METRIC_NAME
#     --ORDER BY 2, 3, PRODUCT_GRP, COMP_CD, METRIC_CD 
# """)


# try:
#     execute_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
#     print(f'\n   -> Execute query... {execute_datetime}')

#     # Get : Sales Monthly DataFrame
#     src_cur.execute(query)
#     rows = src_cur.fetchall()
#     monthly_sales_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])
#     print(f'\n   -> Sales Monthly DataFrame : {monthly_sales_df.shape[0]} rows, {monthly_sales_df.shape[1]} columns')

#     src_cur.close()


# except oracledb.DatabaseError as e:
#     print(f'\nError with Oracle : {e}')


# finally:
#     src_conn.close()
#     print(f'\n{TDMDBPR_db} : Disconnected')

In [28]:
''' Execute Sales Monthly '''


# Input parameter
curr_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
v_start_date = 20240101
print(f'\nParameter input...\n')
print(f'   -> v_start_date: {v_start_date}')


# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
print(f'\n{TDMDBPR_db} : Connected')
src_cur = src_conn.cursor()
query = (f"""
    SELECT /*+PARALLEL(8)*/ 
        SUBSTR(TM_KEY_DAY,1,4) AS TM_KEY_YR, SUBSTR(TM_KEY_DAY,1,6) AS TM_KEY_MTH
        , CASE WHEN REGEXP_LIKE(METRIC_CD, 'CS$|CG$') THEN 'Y2025' ELSE 'Y2024' END VER_FLAG
        , PRODUCT, COMP_CD, METRIC_CD, METRIC_NAME, REMARK, UPD_TM, PPN_TM
        , SUM(CASE WHEN AREA_TYPE = 'P' THEN METRIC_VALUE END) P_ACTUAL
        , SUM(CASE WHEN AREA_TYPE = 'G' THEN METRIC_VALUE END) G
        , SUM(CASE WHEN AREA_TYPE = 'H' THEN METRIC_VALUE END) H
        , SUM(CASE WHEN AREA_TYPE = 'HH' THEN METRIC_VALUE END) HH
        , SUM(CASE WHEN AREA_TYPE = 'CCAA' THEN METRIC_VALUE END) CCAA
        , SUM(CASE WHEN AREA_TYPE = 'CCAATT' THEN METRIC_VALUE END) CCAATT
        , SUM(CASE WHEN AREA_CD = '016' THEN METRIC_VALUE END) "016"
        , SUM(CASE WHEN AREA_CD = '040' THEN METRIC_VALUE END) "040"
        , SUM(CASE WHEN AREA_CD = '080' THEN METRIC_VALUE END) "080"
        , MIN(TM_KEY_DAY) AS START_DAY, MAX(TM_KEY_DAY) AS END_DAY, MAX(LOAD_DATE) LOAD_DATE
    FROM CDSAPPO.DIM_CORP_KPI
    WHERE METRIC_CD NOT LIKE 'Error%'
    AND EXISTS (
        SELECT 1 -->> AREA_CD IN (P, G, H, R, HH, CCAA, CCAATT)
        FROM (
            SELECT DISTINCT ORGID_P AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT ORGID_G AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT ORGID_R AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT ORGID_H AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT ORGID_HH AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT SUBSTR(CCAATT,1,4) AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT CCAATT AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
        ) TMP
        WHERE TMP_CD = AREA_CD )
    AND TM_KEY_DAY >= {v_start_date}
    GROUP BY SUBSTR(TM_KEY_DAY,1,4), SUBSTR(TM_KEY_DAY,1,6), PRODUCT, COMP_CD, METRIC_CD, METRIC_NAME, REMARK, UPD_TM, PPN_TM
    --ORDER BY 2, 3, PRODUCT, COMP_CD, METRIC_CD 
""")


try:
    execute_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
    print(f'\n   -> Execute query... {execute_datetime}')

    # Get : Sales Monthly DataFrame
    src_cur.execute(query)
    rows = src_cur.fetchall()
    monthly_sales_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])
    print(f'\n   -> Sales Monthly DataFrame : {monthly_sales_df.shape[0]} rows, {monthly_sales_df.shape[1]} columns')

    src_cur.close()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()
    print(f'\n{TDMDBPR_db} : Disconnected')


Parameter input...

   -> v_start_date: 20240101



TDMDBPR : Connected

   -> Execute query... 2025-06-20, 10:51:42

   -> Sales Monthly DataFrame : 1268 rows, 22 columns

TDMDBPR : Disconnected


In [29]:
''' Generate CSV file '''

op_dir = 'data'
op_monthly_file = f'sales_monthly_{str_curr_dt}'

monthly_sales_df.to_csv(f'{op_dir}/tmp_{op_monthly_file}.csv', index=False, encoding='utf-8')
print(f'\n   -> Generate "tmp_{op_monthly_file}.csv" successfully')

monthly_sales_df.tail(3)


   -> Generate "tmp_sales_monthly_20250620.csv" successfully


,TM_KEY_YR,TM_KEY_MTH,VER_FLAG,PRODUCT,COMP_CD,METRIC_CD,METRIC_NAME,REMARK,UPD_TM,PPN_TM,...,H,HH,CCAA,CCAATT,016,040,080,START_DAY,END_DAY,LOAD_DATE
1265,2025,202504,Y2025,TVS,TRUE,TB4R001004CG,TVS CMDU Inflow M1 - GEO Channel,KPI Metric 2025,20250620,20250620,...,70722.79,70722.79,70722.79,70722.79,None,None,NaN,20250401,20250428,2025-06-20 08:49:26
1266,2024,202406,Y2025,TVS,TRUE,TB4R001004CG,TVS CMDU Inflow M1 - GEO Channel,KPI Metric 2025,20250304,20250304,...,223326.17,223326.17,223326.17,223326.17,None,None,6728.97,20240601,20240629,2025-03-04 17:49:19
1267,2025,202506,Y2025,POSTPAID,TRUE,TB2R020500CG,Postpaid Inflow M1 B2B : TMH - GEO Channel,KPI Metric 2025,20250620,20250620,...,1247872.40,1247872.40,1247872.40,1247872.40,None,None,18449.00,20250601,20250619,2025-06-20 08:47:41
